In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 16.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633718 sha256=f1211d52a0684c940685d4ade7333268c864ff4536f96ae5e86358b8e4843afd
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
old_df_pizza = pd.read_csv('/content/pizza_ratings.csv')

In [ ]:
df_pizza = old_df_pizza.drop(columns='id')

In [ ]:
df_pizza.head()

,userId,pizza_id,rating
0,1,9275,4.0
1,1,4055,4.0
2,1,9623,4.0
3,1,10667,5.0
4,1,7883,5.0


In [ ]:
df_pizza.isnull().sum()

userId      0
pizza_id    1
rating      0
dtype: int64

In [ ]:
df_pizza["pizza_id"].fillna("111", inplace = True)

In [ ]:
df_pizza.isnull().sum()

userId      0
pizza_id    0
rating      0
dtype: int64

In [ ]:
df_pizza_data = pd.read_csv('/content/data(pizza).csv')

In [ ]:
df_pizza_data.isnull().sum()

pizza_id       0
name           0
description    0
type           0
dtype: int64

In [ ]:
df_pizza_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pizza_id     41 non-null     int64 
 1   name         41 non-null     object
 2   description  41 non-null     object
 3   type         41 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.4+ KB


In [ ]:
df_pizza_data.head()

,pizza_id,name,description,type
0,111,Creamy Tomato Pasta Pizza - Non Veg,tomato|pasta|BBQ pepper chicken|green capsicum...,non-veg
1,343,Moroccan Spice Pasta Pizza - Non Veg,Harissa sauce|Peri Peri chicken|pasta,non-veg
2,575,Chicken Keema Paratha Pizza,chicken keema|cheese,non-veg
3,807,Chicken Salami Gourmet Pizza,chicken salami|bbq chicken|Jalapeno|bocconcini...,non-veg
4,1039,Chicken Pepperoni Gourmet Pizza,Italian Sauce|bocconcini cheese|chicken pepperoni,non-veg


In [ ]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(df_pizza,reader)

In [ ]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  60 

Number of items:  42


In [ ]:
## Perform a gridsearch with SVD
params = {'n_factors' :[20,50,100],
         'reg_all':[0.02,0.05,0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)


In [ ]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.9819799507208659, 'mae': 0.7804650393767565}
{'rmse': {'n_factors': 20, 'reg_all': 0.1}, 'mae': {'n_factors': 20, 'reg_all': 0.1}}


In [ ]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)


In [ ]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.04791685, 1.07156745, 1.0549222 , 1.08329592, 1.06719704]))
('test_mae', array([0.82959876, 0.85739839, 0.83086659, 0.8554207 , 0.8433286 ]))
('fit_time', (0.04606151580810547, 0.09412288665771484, 0.06247735023498535, 0.08278942108154297, 0.04890727996826172))
('test_time', (0.49631762504577637, 0.6925144195556641, 0.6420774459838867, 0.5100002288818359, 0.35706353187561035))
-----------------------
1.064979892001247


In [ ]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([1.03846795, 1.05169584, 1.04219078, 1.0235374 , 1.03605094]))
('test_mae', array([0.81550701, 0.82954755, 0.81978052, 0.80525534, 0.82284153]))
('fit_time', (0.041022300720214844, 0.04414558410644531, 0.039568185806274414, 0.04481363296508789, 0.04035782814025879))
('test_time', (0.3157155513763428, 0.34016966819763184, 0.3215768337249756, 0.3381524085998535, 0.3466188907623291))


1.03838857894067

In [ ]:

svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [ ]:
svd.predict(4,7)

Prediction(uid=4, iid=7, r_ui=None, est=3.32506095975393, details={'was_impossible': False})

# **user** **ratings**

In [ ]:
def pizza_rater(pizza_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            pizz = pizza_df[pizza_df['description'].str.contains(genre)].sample(1)
        else:
            pizz = pizza_df.sample(1)
        print(pizz)
        rating = input('How do you rate this pizza on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_pizza = {'userId':userID,'pizza_id':pizz['pizza_id'].values[0],'rating':rating}
            rating_list.append(rating_one_pizza) 
            num -= 1
    return rating_list

In [ ]:
user_rating = pizza_rater(df_pizza_data,4,'green capsicum')

    pizza_id       name                            description type
26      7535  Farmhouse  onion|green capsicum|tomato|mushrooms  veg
How do you rate this pizza on a scale of 1-5, press n if you have not seen :
5
    pizza_id                  name  \
34     10667  Indi Tandoori Paneer   

                                          description type  
34  Tandoori paneer|green capsicum|red paprika|min...  veg  
How do you rate this pizza on a scale of 1-5, press n if you have not seen :
5
    pizza_id       name                            description type
26      7535  Farmhouse  onion|green capsicum|tomato|mushrooms  veg
How do you rate this pizza on a scale of 1-5, press n if you have not seen :
5
    pizza_id          name           description type
32      9971  Fresh Veggie  onion|green capsicum  veg
How do you rate this pizza on a scale of 1-5, press n if you have not seen :
3


In [ ]:
user_rating

[{'pizza_id': 7535, 'rating': '5', 'userId': 1000},
 {'pizza_id': 10667, 'rating': '5', 'userId': 1000},
 {'pizza_id': 7535, 'rating': '5', 'userId': 1000},
 {'pizza_id': 9971, 'rating': '3', 'userId': 1000}]

In [ ]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(df_pizza,reader)

In [ ]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = df_pizza.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [ ]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [ ]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (df_pizza_data, predicted_score)
list_of_pizza = []
for m_id in df_pizza['pizza_id'].unique():
    list_of_pizza.append( (m_id,svd_.predict(1000,m_id)[3]))

In [ ]:
# order the predictions from highest to lowest rated

ranked_pizza = sorted(list_of_pizza,key=lambda x:x[1],reverse=True)

In [ ]:
# return the top n recommendations using the 
def recommended_pizza(user_ratings,pizza_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = pizza_title_df.loc[pizza_title_df['pizza_id'] == int(rec[0])]['name']
            print('Recommendation # ',idx+1,': ',title,'\n')
            n-= 1
            if n == 0:
                break

recommended_pizza(ranked_pizza,df_pizza_data,5)

Recommendation #  1 :  22    Primavera Gourmet Pizza
Name: name, dtype: object 

Recommendation #  2 :  20    Paneer Paratha Pizza
Name: name, dtype: object 

Recommendation #  3 :  33    Paneer Makhani
Name: name, dtype: object 

Recommendation #  4 :  21    Caprese Gourmet Pizza
Name: name, dtype: object 

Recommendation #  5 :  24    Margherita
Name: name, dtype: object 

